# Q-learning

In this notebook, you will implement Q-learning as described in [Sutton and Barto's book, Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html). We will use the grid ```World``` class from the previous lectures. 

### Install dependencies

In [9]:
! pip install numpy pandas

### Imports

In [10]:
import numpy as np
import random
import sys          # We use sys to get the max value of a float
import pandas as pd # We only use pandas for displaying tables nicely
pd.options.display.float_format = '{:,.3f}'.format

### ```World``` class and globals

The ```World``` is a grid represented as a two-dimensional array of characters where each character can represent free space, an obstacle, or a terminal. Each non-obstacle cell is associated with a reward that an agent gets for moving to that cell (can be 0). The size of the world is _width_ $\times$ _height_ characters.

A _state_ is a tuple $(x,y)$.

An empty world is created in the ```__init__``` method. Obstacles, rewards and terminals can then be added with ```add_obstacle``` and ```add_reward```.

To calculate the next state of an agent (that is, an agent is in some state $s = (x,y)$ and performs and action, $a$), ```get_next_state()```should be called.

In [14]:
# Globals:
ACTIONS = ("up", "down", "left", "right") 

# Rewards, terminals and obstacles are characters:
REWARDS = {" ": 0, ".": 0.1, "+": 10, "-": -10}
TERMINALS = ("+", "-") # Note a terminal should also have a reward assigned
OBSTACLES = ("#")
WORLD_SIZE = 10

# Discount factor
gamma = 1
path = []

# The probability of a random move:
rand_move_probability = 0

class World:  
	def __init__(self, width, height):
		self.width = width
		self.height = height
		# Create an empty world where the agent can move to all cells
		self.grid = np.full((width, height), ' ', dtype='U1')
	
	def add_obstacle(self, start_x, start_y, end_x=None, end_y=None):
		"""
		Create an obstacle in either a single cell or rectangle.
		"""
		if end_x == None: end_x = start_x
		if end_y == None: end_y = start_y
		
		self.grid[start_x:end_x + 1, start_y:end_y + 1] = OBSTACLES[0]

	def add_reward(self, x, y, reward):
		assert reward in REWARDS, f"{reward} not in {REWARDS}"
		self.grid[x, y] = reward

	def add_terminal(self, x, y, terminal):
		assert terminal in TERMINALS, f"{terminal} not in {TERMINALS}"
		self.grid[x, y] = terminal

	def is_obstacle(self, x, y):
		if x < 0 or x >= self.width or y < 0 or y >= self.height:
			return True
		else:
			return self.grid[x ,y] in OBSTACLES 

	def is_terminal(self, x, y):
		return self.grid[x ,y] in TERMINALS

	def get_reward(self, x, y):
		""" 
		Return the reward associated with a given location
		""" 
		return REWARDS[self.grid[x, y]]

	def get_next_state(self, current_state, action):
		"""
		Get the next state given a current state and an action. The outcome can be
		stochastic  where rand_move_probability determines the probability of 
		ignoring the action and performing a random move.
		"""    
		assert action in ACTIONS, f"Unknown acion {action} must be one of {ACTIONS}"
		
		x, y = current_state 
		
		# If our current state is a terminal, there is no next state
		if self.grid[x, y] in TERMINALS:
			return None

		# Check of a random action should be performed:
		if np.random.rand() < rand_move_probability:
			action = np.random.choice(ACTIONS)

		if action == "up":      y -= 1
		elif action == "down":  y += 1
		elif action == "left":  x -= 1
		elif action == "right": x += 1

		# If the next state is an obstacle, stay in the current state
		return (x, y) if not self.is_obstacle(x, y) else current_state


## A simple world

In [12]:
world = World(WORLD_SIZE, WORLD_SIZE)

# Add obstacles
world.add_obstacle(1, 1, 3, 3)
world.add_reward(2, 6, ".")
world.add_reward(5, 2, ".")
world.add_reward(2, 8, ".")
world.add_reward(1, 2, ".")
world.add_reward(9, 9, ".")
world.add_reward(8, 7, ".")
world.add_reward(2, 6, ".")


# Note, that we have to transpose the 2D array (.T) for (x,y)
# to match the convention when displayed
print(world.grid.T)

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' '#' '#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' '.' '#' '#' ' ' '.' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' '#' '#' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' '.' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '.' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' '.' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '.' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '

## Exercise: Q-learning

Implement and test Q-learning. You should be able to base much of your code on your implementation of SARSA. Since Q-learning is an off-policy method, we can use whatever behavior policy we want during training, but the choice of behavioral policy still manners so it is a good idea to balance exploration and exploitation. During testing, we can then use the learnt policy (the target policy).

As for the behavior policy, you can use an simple $\epsilon$-greedy policy, but you can also experiment with alternatives, for instance, optimistic initial values. 

See page 131 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html) for the Q-learning algorithm. 


In [ ]:
# TODO: Implement your code here -- you need a Q-table to keep track of action 
#       value estimates and a policy-function that returns an epsilon greedy 
#       policy based on your estimates. 

# Global variable to keep track of current estimates
# print(len(ACTIONS))
Q = np.full((world.width,world.height,4),0.0)
# print("org:")
# print(Q)

# Our step size / learing rate 
alpha = 0.05 

# Discount factor
gamma = 0.9

epsilon = 0.5

# Episodes to run 
EPISODES = 1000

def epsilon_greedy(x,y):
	if np.random.rand() < epsilon:
		return random.randrange(len(ACTIONS))
	else:
		return np.argmax(Q[x,y,:])

def q_learn_episode(world, policy, start_state):
	# Initialise S:
	current_state = start_state
	steps = 0
	# Loop for each step of episode
	while not world.is_terminal(*current_state):
		steps += 1
		
		# Chose A from S using policy derived from Q
		A = policy(current_state[0],current_state[1])

		# Take Action A observe R, S'
		next_state = world.get_next_state(current_state, ACTIONS[A])
		reward = world.get_reward(*next_state)

		# Q(S,A) <- Q(S,A) + alpha(R + gamme * max_a(Q(S',a)-Q(S,A)))
		S =         (current_state[0],current_state[1],A)
		S_next =    (next_state[0],next_state[1])        
		Q[S] = Q[S] + alpha*(reward + gamma*max(Q[S_next])-Q[S]) 
		
		# S <- S'
		current_state = next_state
	
	return steps
		
def raand_start():
	x = np.random.randint(0,world.width)
	y = np.random.randint(0,world.height)
	if world.is_obstacle(x,y):
		x,y = raand_start()
	return x,y

step_list_q_lern = []

for episode in range(EPISODES):
	# print(f"Episode {episode + 1 }/{EPISODES}:")
	steps = q_learn_episode(world, epsilon_greedy, raand_start())
	step_list_q_lern.append(steps)


Q_best = np.full((world.width,world.height),"               ")
# print("new:")
# print(Q)

for w in range(world.width):
	for h in range(world.height):
		Q_best[w,h] = ACTIONS[np.argmax(Q[w,h,:])]


display(pd.DataFrame(Q_best.T))

import matplotlib.pyplot as plt
plt.title("Q_lern")
plt.plot(step_list_q_lern[:500])
plt.show()

KeyboardInterrupt: 

## Exercise: Compare Q-learning and SARSA

Setup experiments to compare the performance of Q-learning and SARSA. You can use different ```Worlds``` and test different parameter setting, e.g. for $\alpha$ and $\epsilon$. 

In [ ]:
### TODO: Implement your code here

## Optional exercise: Maximization Bias and Double Learning

Below is an implementation of the task shown in Example 6.7 on page 134 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html). There are two states, ```A``` and ```B``` where the agent can perform actions, and a terminal state ```T```. ```A``` and ```B``` have different actions available:

* ```A``` has ```left``` (to ```B```) and ```right``` to the terminal state
* ```B``` has a larger number of actions all leading to a terminal state. 


In [ ]:
# States "A" and "B" have actions while "T" is a terminal state.
STATES = ("A", "B", "T")

class Example67MDP:
	def __init__(self, number_of_B_actions):
		"""
		Create an example and set the number of outgoing actions for state "B"
		(in the book, they do not give a specific number, but merely write that 
		from "B" there "are many possible actions all of which cause immediate
		termination with a reward drawn from a normal distribution with mean 
		-0.1 and variance 1. So, feel free to play with different number of 
		actions in state B)
		
		""" 
		self.number_of_B_actions = number_of_B_actions

	def get_actions(self, state):
		"""
		Returns the set of actions availabe in a given state (a tuple 
		with strings).  
		"""
		assert state in STATES, f"State must be one of {STATES}, not {state}"
		if state == "A":
			return ("left", "right")
		if state == "B":
			return tuple(f"{i}" for i in range(self.number_of_B_actions))
		if state == "T":
			return tuple("N")

	def get_next_state_and_reward(self, state, action):
		"""
		Get the next state and reward given a current state and an action
		"""
		assert state in STATES, f"Unknown state: {state}"
		print(state)
		print(self.get_actions(state))
		assert action in self.get_actions(state), f"Unknown action {action} for state {state}"
		 
		if state == "T":
			raise Exception("The terminal state has no actions and no next state")

		if state == "A":
			if action == "right":
				return "T", 0
			if action == "left":
				return "B", 0

		if state == "B":
			return "T", np.random.normal(loc = -0.1)

	def is_terminal(self, state):
		assert state in STATES, f"Unknown state: {state}"
		return state == "T"


Implement Double Q-learning (see page 136 in [Introduction to Reinforcement Learning](http://incompleteideas.net/book/the-book-2nd.html)) and test it on the ```Example67MDP``` above. Notice, that the number of actions differs between the two states ($\mathcal{A}$(```"A"```) $\neq \mathcal{A}$(```"B"```)), which you have to take into account in your Q-tables. See the code for ```Example67MDP``` above: you can get the set of actions available in a given state by calling ```get_actions(...)``` with the state as argument.

Compare action-value estimates for ```"left"``` and ```"right"``` in state ```"A"```  at different times during learning when using double-Q learning and when using normal Q-learning.

In [ ]:
# Create an instance of Example 6.7 with 10 actions in B
example = Example67MDP(10)

gamma = 1
alpha = 0.05
epsilon = 0.1

# Create two Q-tables (feel free to use your own representation):
# Q1 = np.full((world.width,world.height,3),0.0)
# Q2 = np.full((world.width,world.height,3),0.0)
Q1 = [[0 for _ in range(len(example.get_actions(state)))] for state in STATES]
Q2 = [[0 for _ in range(len(example.get_actions(state)))] for state in STATES]

# Uncomment to disable double-Q-learning:
#Q2 = Q1

# You can use the below policy method if you use the Q1 and Q2 as defined above. 
# If you have done your own representation, you probably have to modify or 
# rewrite the function below:

def e_greedy_dql_policy(state):
  global example
  print(state)
  actions = { a:epsilon/len(example.get_actions(state)) for a in example.get_actions(state) }
  # Do a Q1 + Q2 to do epsilon greedy based on both tables:
  Q = [sum(x) for x in zip(Q1[STATES.index(state)], Q2[STATES.index(state)])]
  actions[example.get_actions(state)[np.argmax(Q)]] = 1 - epsilon + epsilon/len(example.get_actions(state))
  return actions

### TODO: Implement double Q-learning
def double_q_learn_episode(world, policy, start_state):
	# Initialise S:
	current_state = start_state
	
	steps = 0
	
	# Loop for each step of episode
	while not world.is_terminal(*current_state):
		steps += 1
		
		# Chose A from S using policy derived from Q1 +  Q2
		A = policy(current_state)
		print(current_state)

		# Take Action A observe R, S'
		state_next, reward = world.get_next_state_and_reward(current_state,A)

		# Q(S,A) <- Q(S,A) + alpha(R + gamme * max_a(Q(S',a)-Q(S,A)))

		if(np.random.rand() > 0.5):     
			Q1[current_state] = Q1[current_state] + alpha*(reward + gamma*Q2[state_next,np.argmax(Q1[state_next,A])]-Q1[current_state])
		else:
			Q2[current_state] = Q2[current_state] + alpha*(reward + gamma*Q1[state_next,np.argmax(Q2[state_next,A])]-Q2[current_state])
			
		# S <- S'
		current_state = state_next
	
	return steps

for episode in range(EPISODES):
	print(f"Episode {episode + 1 }/{EPISODES}:")
	steps = double_q_learn_episode(example, e_greedy_dql_policy, "A")
	step_list_q_lern.append(steps)


Q1_best = np.full((world.width,world.height),"               ")
# print("new:")
# print(Q)

for w in range(world.width):
	for h in range(world.height):
		Q1_best[w,h] = ACTIONS[np.argmax(Q1[w,h,:])]


display(pd.DataFrame(Q1_best.T))

import matplotlib.pyplot as plt
plt.title("Q_lern")
plt.plot(step_list_q_lern[:100])
plt.show()

Episode 1/1000:
A
A
A
('left', 'right')


AssertionError: Unknown action {'left': 0.9500000000000001, 'right': 0.05} for state A